In [1]:
using DataStructures
using Parameters
@with_kw type Problem
    V::Int32 = 0
    E::Int32 = 0
    R::Int32 = 0
    C::Int32 = 0
    X::Int32 = 0
    video_sizes::Array{Int32} = Array{Int32}(0)
    endpoint_to_cache_to_latency::DefaultDict{Int32, Dict{Int32, Int32}} = DefaultDict{Int32, Dict{Int32, Int32}}(() -> Dict{Int32, Int32}())
    video_endpoint_popularity::Array{Tuple{Int32, Int32, Int32}} = Array{Tuple{Int32, Int32, Int32}}(0) 
    
    cache_to_videos::Array{IntSet} = Array{IntSet}(0)
    cache_size::Array{Int32} = Array{Int32}(0)
end

Problem

In [2]:
function read_problem(filename)
    p = Problem()
    open(filename) do f
        p.V, p.E, p.R, p.C, p.X = readdlm(IOBuffer(readline(f)), ' ', Int32)
        p.video_sizes = readdlm(IOBuffer(readline(f)), ' ', Int32)[1, :]
        p.endpoint_to_cache_to_latency = DefaultDict{Int32, Dict{Int32, Int32}}(() -> Dict{Int32, Int32}())
        for e = 1:p.E
            dc_latency, K = readdlm(IOBuffer(readline(f)), ' ', Int32)
            for k = 1:K
                c, latency = readdlm(IOBuffer(readline(f)), ' ', Int32)
                p.endpoint_to_cache_to_latency[e][c] = latency
            end
        end

        p.video_endpoint_popularity = Array{Tuple{Int32, Int32, Int32}}(p.R) 
        for r = 1:p.R
            v, e, pop = readdlm(IOBuffer(readline(f)), ' ', Int32)
            p.video_endpoint_popularity[r] = v, e, pop
        end
    end
    return p
end

read_problem (generic function with 1 method)

In [5]:
function solve(p)
    sort!(p.video_endpoint_popularity, by=x -> x[3], rev=true)
    p.cache_to_videos = [IntSet() for _ in 1:p.C]
    p.cache_size = fill(Int32(0), p.C)
    for vep in p.video_endpoint_popularity
        v, e, pop = vep
        c_min, l_min = Inf, Inf
        for c_l in p.endpoint_to_cache_to_latency[e + 1]
            c, l = c_l
            if p.video_sizes[v + 1] + p.cache_size[c + 1] <= p.X
                if l_min > l
                    c_min, l_min = c, l
                end
            end
        end
        if l_min != Inf
            p.cache_size[c_min + 1] += p.video_sizes[v + 1]
            push!(p.cache_to_videos[c_min + 1], v + 1)
        end
    end
end

solve (generic function with 1 method)

 at In[3]:2 overwritten at In[5]:2.


In [6]:
p = read_problem("me_at_the_zoo.in")
solve(p)

In [7]:
p.cache_to_videos

10-element Array{IntSet,1}:
 IntSet([1, 6, 9, 83, 100])   
 IntSet([1, 2, 3, 6, 66])     
 IntSet([1, 3, 17, 20])       
 IntSet([1, 5, 6, 11, 17, 31])
 IntSet([1, 11, 18, 33, 82])  
 IntSet([1, 8, 14])           
 IntSet([1, 3, 5])            
 IntSet([6, 17, 22, 31, 75])  
 IntSet([1, 4, 7, 9, 11, 17]) 
 IntSet([1, 2, 5, 9, 17])     

In [8]:
[ [p.video_sizes[j] for j in i ] for i in p.cache_to_videos]

10-element Array{Array{Int32,1},1}:
 Int32[20,3,40,12,5]  
 Int32[20,11,50,3,16] 
 Int32[20,50,1,27]    
 Int32[20,5,3,4,1,46] 
 Int32[20,4,37,15,4]  
 Int32[20,32,27]      
 Int32[20,50,5]       
 Int32[3,1,33,46,12]  
 Int32[20,26,6,40,4,1]
 Int32[20,11,5,40,1]  